# Данные телеметрии болида Л. Хэмилтона на гонке в Монако в 2011 году

In [6]:
import urllib
import pandas
import random

url = 'https://raw.githubusercontent.com/MargoILIrita/data_analyse_labs/master/telemetry.csv'
data = pandas.read_csv(url, delimiter='\t', index_col=['timestamp'],
                       usecols=['timestamp','nEngine',
                                'rThrottlePedal','pBrakeF','gLat','gLong'])
lapSpeed = pandas.read_csv(url, delimiter='\t', index_col=['Lap', 'sLap'],
                       usecols=['Lap', 'sLap','vCar','NGear','timestamp'])
lapSpeed.head(10)

timestamp  NGear   vCar
Lap sLap                        
1   3046  14:02:50      1    0.0
    1     14:02:54      1   95.1
    73    14:02:56      3  175.6
    228   14:03:00      1   67.5
    262   14:03:02      1   75.9
    322   14:03:04      3  153.1
    440   14:03:06      5  211.8
    557   14:03:08      6  228.0
    685   14:03:10      4  169.3
    758   14:03:12      3  126.9

## Смена индексов

In [8]:
lapSpeed.swaplevel(0,1).sort_index(0).head(10)

timestamp  NGear  vCar
sLap Lap                       
0    23   14:32:55      1  59.5
     23   14:32:56      1  59.8
     23   14:33:00      1   0.0
     23   14:33:01      1   0.0
     23   14:33:02      1   0.0
     23   14:33:03      1   0.0
     23   14:33:04      1   0.0
     23   14:33:05      1   0.0
     23   14:33:06      1   0.0
     23   14:33:07      1   0.0

In [9]:
lapSpeed.T.sort_index(0)

Lap              1                                                     \
sLap           3046      1         73        228       262       322    
NGear             1         1         3         1         1         3   
timestamp  14:02:50  14:02:54  14:02:56  14:03:00  14:03:02  14:03:04   
vCar              0      95.1     175.6      67.5      75.9     153.1   

Lap                                                  ...           75  \
sLap           440       557       685       758     ...         0      
NGear             5         6         4         3    ...            0   
timestamp  14:03:06  14:03:08  14:03:10  14:03:12    ...     16:15:56   
vCar          211.8       228     169.3     126.9    ...            0   

Lap                                                                    \
sLap           0         0         0         0         0         0      
NGear             0         0         0         0         0         0   
timestamp  16:15:58  16:15:59  16:16:02  16:16:04  16:16:06  16:16:08   
vCar              0         0         0         0         0         0   

Lap                                      
sLap           0         0         0     
NGear             0         0         0  
timestamp  16:16:10  16:16:11  16:16:14  
vCar              0         0         0  

[3 rows x 5476 columns]

## Добавление и удаление индексов

In [10]:
lapSpeed.reset_index(level=1).sort_index(0).head(10)

,sLap,timestamp,NGear,vCar
Lap,,,,
1,3046,14:02:50,1,0.0
1,1,14:02:54,1,95.1
1,73,14:02:56,3,175.6
1,228,14:03:00,1,67.5
1,262,14:03:02,1,75.9
1,322,14:03:04,3,153.1
1,440,14:03:06,5,211.8
1,557,14:03:08,6,228.0
1,685,14:03:10,4,169.3


In [11]:
lapSpeed.reset_index().set_index(['Lap','NGear']).sort_index(0).head(10)

sLap timestamp  vCar
Lap NGear                      
1   1      3046  14:02:50   0.0
    1         1  14:02:54  95.1
    1       228  14:03:00  67.5
    1       262  14:03:02  75.9
    1      1146  14:03:24  69.4
    1      1188  14:03:26  91.7
    1      1238  14:03:28  75.9
    1      1266  14:03:30  47.6
    1      1291  14:03:32  55.3
    1      1339  14:03:34  75.7

## Объединение данных о скорости с другими данными телеметрии

In [12]:
lapSpeed = lapSpeed.reset_index().set_index('timestamp')
data = data.merge(lapSpeed,on='timestamp')
data.head(10)

,nEngine,rThrottlePedal,pBrakeF,gLat,gLong,Lap,sLap,NGear,vCar
timestamp,,,,,,,,,
14:02:50,13449,8,10,0,0,1,3046,1,0.0
14:02:54,15156,81,1,0,1,1,1,1,95.1
14:02:56,14953,37,5,0,0,1,73,3,175.6
14:03:00,9576,0,13,1,0,1,228,1,67.5
14:03:02,16341,63,1,0,1,1,262,1,75.9
14:03:04,16610,100,1,0,1,1,322,3,153.1
14:03:06,17025,100,1,0,1,1,440,5,211.8
14:03:08,16034,100,2,1,0,1,557,6,228.0
14:03:10,14306,4,14,-1,-1,1,685,4,169.3


## Среднее значение скорости в зависимости от круга и передачи

In [13]:
data.groupby(['Lap','NGear'])['vCar'].mean().head(10)

Lap  NGear
1    1         70.589474
     2        121.787500
     3        148.350000
     4        185.325000
     5        211.800000
     6        235.350000
     7        264.600000
2    1         79.450000
     2        118.042857
     3        163.250000
Name: vCar, dtype: float64

## Использование pivot
### Значение скорости на разных передачах в разных кругах

In [15]:
df = data.drop_duplicates(['NGear', 'Lap'])
df.pivot('NGear', 'Lap','vCar').fillna(0).head(10)

Lap,1,2,3,4,5,6,7,8,9,10,...,66,67,68,69,70,71,72,73,74,75
NGear,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,21.8,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0
1,0.0,96.1,67.9,100.3,81.8,77.9,77.0,68.4,79.1,77.0,...,61.8,52.4,0.0,51.2,100.1,115.6,71.8,71.3,28.9,0.0
2,107.9,117.5,135.2,133.1,120.1,120.9,117.4,136.9,134.3,125.4,...,88.9,98.5,59.5,105.8,102.8,120.4,143.4,115.8,79.4,0.0
3,175.6,191.9,183.8,160.0,166.7,141.5,156.2,169.3,173.3,159.3,...,139.6,164.9,128.5,155.5,163.9,180.2,108.2,90.1,81.1,0.0
4,169.3,193.9,187.8,189.2,197.5,172.5,203.0,202.4,192.8,193.4,...,194.7,198.0,178.5,190.5,196.8,203.3,198.6,158.8,152.5,0.0
5,211.8,0.0,0.0,0.0,206.5,211.6,221.2,223.5,222.5,0.0,...,194.7,218.6,211.3,218.8,230.2,224.2,223.8,202.2,56.5,0.0
6,228.0,235.7,233.8,236.1,239.7,250.0,242.1,243.2,244.0,234.9,...,224.7,230.9,233.9,220.2,237.8,0.0,0.0,249.9,86.5,0.0
7,264.6,246.9,265.7,270.4,266.5,266.3,261.0,256.3,262.7,267.2,...,0.0,169.6,0.0,266.2,266.4,268.3,264.2,256.0,83.4,0.0


### Значения оборотов двигателя на разных кругах с учётом процента нажатия педали газа

In [17]:
df = data.drop_duplicates(['Lap','rThrottlePedal'])
df = df.pivot_table(index=['Lap'], columns='rThrottlePedal',values='nEngine').fillna(0)
df.head(10)

rThrottlePedal,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
Lap,,,,,,,,,,,,,,,,,,,,,
1,9576.0,0.0,15431.0,11720.0,14306.0,0.0,0.0,0.0,13449.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16610.0
2,14343.0,13877.0,15902.0,14137.0,0.0,0.0,0.0,0.0,0.0,11856.0,...,16604.0,0.0,0.0,0.0,17055.0,0.0,0.0,0.0,0.0,17253.0
3,13373.0,15743.0,0.0,0.0,0.0,14492.0,0.0,0.0,0.0,0.0,...,0.0,14994.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17318.0
4,13872.0,15995.0,0.0,0.0,0.0,16424.0,14810.0,0.0,0.0,0.0,...,17719.0,0.0,0.0,17881.0,0.0,0.0,16984.0,0.0,0.0,17408.0
5,15185.0,15568.0,15837.0,0.0,0.0,14281.0,9231.0,0.0,14123.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16629.0,0.0,0.0,16483.0
6,15416.0,15054.0,17062.0,0.0,14288.0,0.0,0.0,0.0,6425.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17801.0
7,15170.0,15343.0,0.0,0.0,15104.0,11088.0,0.0,0.0,0.0,12766.0,...,0.0,0.0,0.0,0.0,15891.0,0.0,0.0,14473.0,16985.0,17706.0
8,9081.0,14560.0,0.0,0.0,14539.0,15212.0,16696.0,14391.0,12857.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16811.0
9,14475.0,14433.0,10753.0,0.0,0.0,15570.0,14903.0,15166.0,14056.0,0.0,...,17751.0,0.0,0.0,0.0,0.0,16694.0,0.0,0.0,0.0,16794.0
